# Run QARTOD Test on Locally Saved Data

In this notebook we will load locally saved data from the interim data folder, extract QARTOD test parameters from spreadsheets on the OOI GitHub, run the QARTOD climatology and gross range tests on the imported data, and save the test results to the processed data folder.

More info about QARTOD tests and the ioos_qc module can be found from the [Integrated Ocean Observing System website](https://ioos.noaa.gov/project/qartod/) and [Python module documentation](https://ioos.github.io/ioos_qc/), respectively.

### Import modules used in this notebook

In [ ]:
# Import libraries
import os
import re
import gc
import io
import ast
import pandas as pd
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings("ignore")
import sys

# Import IOOS module with QARTOD tests
import ioos_qc

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

In [ ]:
# Import OOINet library - might delete
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\OOINet") # this is what was missing from the steps I followed to install ooinet and ooi-data-explorations as local dev repo
from ooinet import M2M
from ooinet.Instrument.common import process_file

In [ ]:
# Import functions from ooi-data-explorations library - might delete
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\ooi-data-explorations\\python") # why did the initial install not include this?
from ooi_data_explorations.uncabled.process_dosta import dosta_datalogger
from ooi_data_explorations.combine_data import combine_datasets

### Load locally saved data

In [ ]:
# Use reference designator, data stream, and method to build file name and path for chosen dataset

site = "CP01CNSM"                                   # Coastal Pioneer Array (NES) - Central Surface Mooring
node = "MFD37"                                      # 
sensor = "03-CTDBPD000"                             # CTD Bottom-pumped
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

refdes = '-'.join((site,node,sensor))

In [ ]:
# Build filename and path to interim data
# This whole cell is something that would be good to turn into a function in the source code since we also repeat a similar process in notebook 01

dev_filename = '-'.join(('dev',refdes,method,stream))+'.nc'             # build ds_dev filename from dataset attributes
prod_filename = '-'.join(('prod',refdes,method,stream))+'.nc'           # ds_prod filename from dataset attribute

interim_data = os.path.relpath('../data/interim')                       # path to interim data folder from notebook folder

dev_path=os.path.join(interim_data, dev_filename)                       # build full relative path with ds_dev filename
prod_path=os.path.join(interim_data, prod_filename)                     # repeat for ds_prod

In [ ]:
# Load data from .nc files
ds_prod = xr.open_dataset(prod_path)
ds_dev = xr.open_dataset(dev_path)

### Collect test QARTOD value tables

### Run QARTOD tests locally

### Save test results to processed data folder